[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/finance-data-analysis/blob/main/3.3%20%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B8%88%EC%9C%B5%20%EA%B0%9C%EB%B3%84%EC%A2%85%EB%AA%A9%20%EC%88%98%EC%A7%91-output.ipynb)

## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 하이브 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd

## 수집할 URL 정하기

In [2]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "352820"
# item_name = "빅히트"

item_code = "326030"
item_name = "SK바이오팜"
page_no = 1

# 종목 URL 만들기
url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
url

'https://finance.naver.com/item/sise_day.nhn?code=326030&page=1'

## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [3]:
import requests

headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

response = requests.get(url, headers=headers)

In [4]:
#response.text

## BeautifulSoup 을 통한 table 태그 찾기

* [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [5]:
from bs4 import BeautifulSoup as bs

html = bs(response.text, "lxml")
table = html.select("table")
len(table)

2

In [6]:
type(table)

list

## pandas 코드 한 줄로 데이터 수집하기

In [7]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
table = pd.read_html(str(table))

In [8]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[0]

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021.09.08,116500.0,4500.0,121000.0,121000.0,116500.0,334963.0
2,2021.09.07,121000.0,500.0,121500.0,123000.0,120000.0,140250.0
3,2021.09.06,121500.0,3500.0,125000.0,125000.0,121500.0,180646.0
4,2021.09.03,125000.0,3000.0,123500.0,126500.0,122500.0,270216.0
5,2021.09.02,122000.0,2000.0,125000.0,125000.0,122000.0,191733.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021.09.01,124000.0,1500.0,126000.0,126000.0,123500.0,206858.0


In [9]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
temp = table[0].dropna()
temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.09.08,116500.0,4500.0,121000.0,121000.0,116500.0,334963.0
2,2021.09.07,121000.0,500.0,121500.0,123000.0,120000.0,140250.0
3,2021.09.06,121500.0,3500.0,125000.0,125000.0,121500.0,180646.0
4,2021.09.03,125000.0,3000.0,123500.0,126500.0,122500.0,270216.0
5,2021.09.02,122000.0,2000.0,125000.0,125000.0,122000.0,191733.0
9,2021.09.01,124000.0,1500.0,126000.0,126000.0,123500.0,206858.0
10,2021.08.31,125500.0,4000.0,122500.0,125500.0,122000.0,342891.0
11,2021.08.30,121500.0,0.0,122000.0,123000.0,120500.0,146824.0
12,2021.08.27,121500.0,4500.0,117500.0,123000.0,117500.0,415457.0
13,2021.08.26,117000.0,1000.0,118000.0,118500.0,116500.0,131690.0


## 페이지별 데이터 수집 함수 만들기

In [10]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """ 
    url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
    
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

    response = requests.get(url, headers=headers)
    html = bs(response.text, "lxml")
    table = html.select("table")
    table = pd.read_html(str(table))
    temp = table[0].dropna()
    return temp

In [11]:
# 함수가 잘 만들어졌는지 확인하기
get_day_list(item_code, 1)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.09.08,116500.0,4500.0,121000.0,121000.0,116500.0,334963.0
2,2021.09.07,121000.0,500.0,121500.0,123000.0,120000.0,140250.0
3,2021.09.06,121500.0,3500.0,125000.0,125000.0,121500.0,180646.0
4,2021.09.03,125000.0,3000.0,123500.0,126500.0,122500.0,270216.0
5,2021.09.02,122000.0,2000.0,125000.0,125000.0,122000.0,191733.0
9,2021.09.01,124000.0,1500.0,126000.0,126000.0,123500.0,206858.0
10,2021.08.31,125500.0,4000.0,122500.0,125500.0,122000.0,342891.0
11,2021.08.30,121500.0,0.0,122000.0,123000.0,120500.0,146824.0
12,2021.08.27,121500.0,4500.0,117500.0,123000.0,117500.0,415457.0
13,2021.08.26,117000.0,1000.0,118000.0,118500.0,116500.0,131690.0


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [12]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []

while True:
    print(page_no)
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)
    
    page_no = page_no + 1
    time.sleep(0.1)
    
    #if temp.shape[0] < 10:
    if page_no > 10:
        break;

1
2
3
4
5
6
7
8
9
10


In [13]:
len(item_list)

10

## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [14]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list)

<img src="https://pandas.pydata.org/docs/_images/08_concat_row.svg">

In [15]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.09.08,116500.0,4500.0,121000.0,121000.0,116500.0,334963.0
2,2021.09.07,121000.0,500.0,121500.0,123000.0,120000.0,140250.0
3,2021.09.06,121500.0,3500.0,125000.0,125000.0,121500.0,180646.0
4,2021.09.03,125000.0,3000.0,123500.0,126500.0,122500.0,270216.0
5,2021.09.02,122000.0,2000.0,125000.0,125000.0,122000.0,191733.0


In [16]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
9,2021.04.23,118500.0,1000.0,116000.0,119000.0,115500.0,283501.0
10,2021.04.22,117500.0,2500.0,120500.0,121000.0,117000.0,383676.0
11,2021.04.21,120000.0,1000.0,118500.0,121000.0,117500.0,654108.0
12,2021.04.20,119000.0,5500.0,114500.0,119500.0,114000.0,727766.0
13,2021.04.19,113500.0,2000.0,111000.0,115500.0,110500.0,622540.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [17]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
df["종목코드"] = item_code
df["종목명"] = item_name
df

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2021.09.08,116500.0,4500.0,121000.0,121000.0,116500.0,334963.0,326030,SK바이오팜
2,2021.09.07,121000.0,500.0,121500.0,123000.0,120000.0,140250.0,326030,SK바이오팜
3,2021.09.06,121500.0,3500.0,125000.0,125000.0,121500.0,180646.0,326030,SK바이오팜
4,2021.09.03,125000.0,3000.0,123500.0,126500.0,122500.0,270216.0,326030,SK바이오팜
5,2021.09.02,122000.0,2000.0,125000.0,125000.0,122000.0,191733.0,326030,SK바이오팜
...,...,...,...,...,...,...,...,...,...
9,2021.04.23,118500.0,1000.0,116000.0,119000.0,115500.0,283501.0,326030,SK바이오팜
10,2021.04.22,117500.0,2500.0,120500.0,121000.0,117000.0,383676.0,326030,SK바이오팜
11,2021.04.21,120000.0,1000.0,118500.0,121000.0,117500.0,654108.0,326030,SK바이오팜
12,2021.04.20,119000.0,5500.0,114500.0,119500.0,114000.0,727766.0,326030,SK바이오팜


## 컬럼 순서 변경하기

In [18]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.
df.columns

Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '종목코드', '종목명'], dtype='object')

In [19]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]

## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [20]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
df = df.drop_duplicates()
df.shape

(100, 9)

## 기술통계값 구하기

In [21]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000e+02
mean,119470.000000,1970.000000,119585.000000,121520.000000,117650.000000,3.449243e+05
std,6823.466639,1563.181165,6895.341349,7025.897261,6644.090562,2.158254e+05
min,104500.000000,0.000000,103500.000000,105500.000000,102500.000000,1.254870e+05
25%,113500.000000,500.000000,113875.000000,115000.000000,112000.000000,2.069660e+05
50%,120500.000000,1750.000000,120750.000000,122500.000000,118750.000000,2.680610e+05
75%,124000.000000,3000.000000,124500.000000,126500.000000,122625.000000,4.078948e+05
max,135000.000000,7000.000000,134500.000000,137500.000000,132000.000000,1.286901e+06


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [22]:
# 거래량의 과학적 기수법 읽기
1.000000e+02 # (10의 제곱)

100.0

In [23]:
4.701523e+05 # (10의 5승)

470152.3

In [24]:
4.701523 * (10 ** 5)

470152.3

## 최근 날짜 구해서 파일명 만들기

In [25]:
# 날짜 column의 첫 row값 확인
date = df.iloc[0]["날짜"]
date

'2021.09.08'

In [26]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
file_name = f"data/{item_name}_{item_code}_{date}.csv"
file_name

'data/SK바이오팜_326030_2021.09.08.csv'

## 파일로 저장하기

In [28]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.
df.to_csv(file_name, index=False)

In [29]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,326030,SK바이오팜,2021.09.08,116500.0,4500.0,121000.0,121000.0,116500.0,334963.0
1,326030,SK바이오팜,2021.09.07,121000.0,500.0,121500.0,123000.0,120000.0,140250.0
2,326030,SK바이오팜,2021.09.06,121500.0,3500.0,125000.0,125000.0,121500.0,180646.0
3,326030,SK바이오팜,2021.09.03,125000.0,3000.0,123500.0,126500.0,122500.0,270216.0
4,326030,SK바이오팜,2021.09.02,122000.0,2000.0,125000.0,125000.0,122000.0,191733.0
...,...,...,...,...,...,...,...,...,...
95,326030,SK바이오팜,2021.04.23,118500.0,1000.0,116000.0,119000.0,115500.0,283501.0
96,326030,SK바이오팜,2021.04.22,117500.0,2500.0,120500.0,121000.0,117000.0,383676.0
97,326030,SK바이오팜,2021.04.21,120000.0,1000.0,118500.0,121000.0,117500.0,654108.0
98,326030,SK바이오팜,2021.04.20,119000.0,5500.0,114500.0,119500.0,114000.0,727766.0


## 전체 과정을 하나의 함수로 만들기

In [30]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    # web page 시작번호
    page_no = 1
    # 데이터를 저장할 빈 변수 선언
    item_list = []

    while True:
        temp = get_day_list(item_code, page_no)
        item_list.append(temp)

        page_no = page_no + 1
        time.sleep(0.1)

        #if temp.shape[0] < 10:
        if page_no > 10:
            break
            
    df = pd.concat(item_list)
    df["종목코드"] = item_code
    df["종목명"] = item_name
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols]

    return df

In [31]:
item_code = "352820"
item_name = "하이브"
# 빅히트의 사명이 하이브로 변경되었습니다.

# item_code = "326030"
# item_name = "SK바이오팜"

get_item_list(item_code, item_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,352820,하이브,2021.09.08,280000.0,6000.0,286000.0,287000.0,278500.0,129676.0
2,352820,하이브,2021.09.07,286000.0,500.0,286500.0,288000.0,284500.0,92619.0
3,352820,하이브,2021.09.06,286500.0,5500.0,280500.0,288000.0,279500.0,129959.0
4,352820,하이브,2021.09.03,281000.0,500.0,284500.0,284500.0,279500.0,115176.0
5,352820,하이브,2021.09.02,280500.0,5500.0,287000.0,287000.0,280500.0,125291.0
...,...,...,...,...,...,...,...,...,...
9,352820,하이브,2021.04.23,258500.0,3500.0,256000.0,261500.0,251500.0,285699.0
10,352820,하이브,2021.04.22,255000.0,13000.0,246000.0,256000.0,245500.0,446672.0
11,352820,하이브,2021.04.21,242000.0,0.0,242000.0,246500.0,241000.0,231381.0
12,352820,하이브,2021.04.20,242000.0,500.0,241000.0,247500.0,239500.0,215412.0


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [32]:
# 도움말 보기
get_item_list?


In [33]:
# 소스코드 보기
get_item_list??


##  [Q&A] 앙상블 학습

이론 : https://formal.hknu.ac.kr/handson-ml2/slides/handson-ml2-07-slides.pdf

실습 : https://injo.tistory.com/22